In [1]:
import pandas as pd
pattern_n = pd.read_csv('patterns_nl.txt', delimiter = '\t')
pattern_n

,pattern,iate_%,headword,quality
0,NOUN,0.34,0,1
1,ADJ NOUN,0.10,1,1
2,NOUN ADP NOUN,0.03,0,1
3,ADV,0.03,0,0
4,NOUN ADP DET NOUN,0.03,1,1
5,ADJ,0.02,0,1
6,NOUN ADP ADJ NOUN,0.01,0,1
7,ADP NOUN,0.01,1,1
8,ADJ ADJ NOUN,0.01,2,1
9,ADP,0.01,0,1


In [1]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
stanza.download('nl')
nlp = stanza.Pipeline(lang='nl', processors='tokenize,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to /Users/hanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2021-09-14 13:17:43 INFO: Downloading default packages for language: nl (Dutch)...
2021-09-14 13:17:45 INFO: File exists: /Users/hanh/stanza_resources/nl/default.zip.
2021-09-14 13:17:48 INFO: Finished downloading models and saved to /Users/hanh/stanza_resources.
2021-09-14 13:17:48 INFO: Loading these models for language: nl (Dutch):
| Processor | Package |
-----------------------
| tokenize  | alpino  |
| pos       | alpino  |
| lemma     | alpino  |

2021-09-14 13:17:48 INFO: Use device: cpu
2021-09-14 13:17:48 INFO: Loading: tokenize
2021-09-14 13:17:48 INFO: Loading: pos
2021-09-14 13:17:48 INFO: Loading: lemma
2021-09-14 13:17:48 INFO: Done loading processors!


1. ADJ NOUN -1

In [3]:
def get_term_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'ADJ'):
                        terms.append(' '.join([b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

2. NOUN ADP NOUN - 0

In [4]:
def get_term_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i+1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN')and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'ADP') and (a1_word.upos == 'NOUN')):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]))                               
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

3. NOUN ADP DET NOUN - 1

In [5]:
def get_term_noun_adp_det_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                #adj_noun_adj_noun
                if (b_pos != 0) and (i + 1 < len(tokens)) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and  (len(nlp(str(tokens[i+1])).sentences) > 0): 
                    a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                    a_word = nlp(str(tokens[i])).sentences[0].words[0]
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    # Check vị trí b_pos - 1: terms.append()
                    if (c_word.text != 'None') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (c_word.upos == 'ADP') and ((b_word.upos == 'NOUN') and (a_word.upos == 'DET') and (a1_word.upos == 'NOUN')):
                        terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]))
                                
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

4. NOUN ADP ADJ NOUN - 0

In [6]:
def get_term_noun_adp_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())
        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[i+2])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0] 
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (a2_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.upos == 'ADJ') and (a2_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] +[a2_word.text]))   
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms    

5. ADP NOUN - 1

In [7]:
def get_term_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'ADP'):
                        terms.append(' '.join([b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

6. ADJ ADJ NOUN - 2

In [8]:
def get_term_adj_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    #ADJ ADJ NOUN              
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [2]:
def term_evaluation(domain_path, preds_path, rule=None):
    groundtruth = pd.read_csv(domain_path, sep='	', engine='python',header=None)
    gt = list(groundtruth[0])
    predictions = pkl.load(open(preds_path, 'rb'))
    if rule == 'adj_noun':
        preds =  get_term_adj_noun(predictions)
    elif rule == 'noun_adp_noun':
        preds =  get_term_noun_adp_noun(predictions)
    elif rule == 'noun_adp_det_noun':
        preds = get_term_noun_adp_det_noun(predictions)
    elif rule == 'noun_adp_adj_noun':
        preds = get_term_noun_adp_adj_noun(predictions)
    elif rule == 'adp_noun':
        preds = get_term_adp_noun(predictions)
    elif rule == 'adj_adj_noun':
        preds = get_term_adj_adj_noun(predictions)
    else:
        preds =  get_term_(predictions)
    stop_words = set(stopwords.words('dutch'))
    pred_terms =  set(preds) - set(stop_words)
    pred_terms = [x for x in pred_terms if len(x)>1]
    pred_terms = [x.lower().strip() for x in pred_terms]
    pred_terms = [re.sub(' -','-', x) for x in pred_terms]
    pred_terms = [re.sub('- ','-', x) for x in pred_terms]
    pred_terms = [re.sub('\(','', x) for x in pred_terms]
    pred_terms = [re.sub('\/','', x) for x in pred_terms]
    precision, recall, f1 = evaluation_metrics(pred_terms, gt)
    return precision, recall, f1

In [12]:
path = '/Users/hanhtran/Documents/terminology-extraction/ACTER/'
path1 = '/Users/hanhtran/Documents/terminology-extraction/patterns/nl_domains/corp_wind/'
path2 = '/Users/hanhtran/Documents/terminology-extraction/patterns/nl_domains/corp_equi/'
path3 = '/Users/hanhtran/Documents/terminology-extraction/patterns/nl_domains/equi_wind/'
domain_paths = [path+'nl/equi/annotations/equi_nl_terms.ann',path+'nl/equi/annotations/equi_nl_terms_nes.ann',
                path+'nl/wind/annotations/wind_nl_terms.ann',path+'nl/wind/annotations/wind_nl_terms_nes.ann',
                path+'nl/corp/annotations/corp_nl_terms.ann',path+'nl/corp/annotations/corp_nl_terms_nes.ann',
               ]
preds_paths = [path1+'ann_bert_equi.pkl', path1+'nes_bert_equi.pkl', 
               path2+'ann_bert_wind.pkl', path2+'nes_bert_wind.pkl',
               path3+'ann_bert_corp.pkl', path3+'nes_bert_corp.pkl']
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p)
    results.append([p, f1])
raw_res = pd.DataFrame(results,columns=['DOMAIN','NON_PATTERN'])
raw_res

,DOMAIN,NON_PATTERN
0,/Users/hanhtran/Documents/terminology-extracti...,45.76
1,/Users/hanhtran/Documents/terminology-extracti...,51.86
2,/Users/hanhtran/Documents/terminology-extracti...,37.99
3,/Users/hanhtran/Documents/terminology-extracti...,42.35
4,/Users/hanhtran/Documents/terminology-extracti...,24.24
5,/Users/hanhtran/Documents/terminology-extracti...,30.77


In [13]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_noun')
    results.append([p, f1])
adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN'])
adj_noun

,DOMAIN,ADJ_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,44.85
1,/Users/hanhtran/Documents/terminology-extracti...,50.61
2,/Users/hanhtran/Documents/terminology-extracti...,35.05
3,/Users/hanhtran/Documents/terminology-extracti...,39.66
4,/Users/hanhtran/Documents/terminology-extracti...,23.90
5,/Users/hanhtran/Documents/terminology-extracti...,30.51


In [14]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_noun')
    results.append([p, f1])
noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN'])
noun_adp_noun

,DOMAIN,NOUN_ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,44.77
1,/Users/hanhtran/Documents/terminology-extracti...,50.90
2,/Users/hanhtran/Documents/terminology-extracti...,36.20
3,/Users/hanhtran/Documents/terminology-extracti...,40.40
4,/Users/hanhtran/Documents/terminology-extracti...,23.88
5,/Users/hanhtran/Documents/terminology-extracti...,30.32


In [15]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_det_noun')
    results.append([p, f1])
noun_adp_det_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_DET_NOUN'])
noun_adp_det_noun

,DOMAIN,NOUN_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,45.76
1,/Users/hanhtran/Documents/terminology-extracti...,51.86
2,/Users/hanhtran/Documents/terminology-extracti...,37.95
3,/Users/hanhtran/Documents/terminology-extracti...,42.33
4,/Users/hanhtran/Documents/terminology-extracti...,24.24
5,/Users/hanhtran/Documents/terminology-extracti...,30.75


In [16]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_adj_noun')
    results.append([p, f1])
noun_adp_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_ADJ_NOUN'])
noun_adp_adj_noun

,DOMAIN,NOUN_ADP_ADJ_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,45.62
1,/Users/hanhtran/Documents/terminology-extracti...,51.70
2,/Users/hanhtran/Documents/terminology-extracti...,37.30
3,/Users/hanhtran/Documents/terminology-extracti...,41.64
4,/Users/hanhtran/Documents/terminology-extracti...,24.27
5,/Users/hanhtran/Documents/terminology-extracti...,30.70


In [17]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adp_noun')
    results.append([p,f1])
adp_noun = pd.DataFrame(results,columns=['DOMAIN','ADP_NOUN'])
adp_noun

,DOMAIN,ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,43.43
1,/Users/hanhtran/Documents/terminology-extracti...,49.27
2,/Users/hanhtran/Documents/terminology-extracti...,34.15
3,/Users/hanhtran/Documents/terminology-extracti...,38.46
4,/Users/hanhtran/Documents/terminology-extracti...,23.01
5,/Users/hanhtran/Documents/terminology-extracti...,29.69


In [18]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_adj_noun')
    results.append([p, f1])
adj_adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_ADJ_NOUN'])
adj_adj_noun

,DOMAIN,ADJ_ADJ_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,45.57
1,/Users/hanhtran/Documents/terminology-extracti...,51.71
2,/Users/hanhtran/Documents/terminology-extracti...,37.50
3,/Users/hanhtran/Documents/terminology-extracti...,41.83
4,/Users/hanhtran/Documents/terminology-extracti...,24.15
5,/Users/hanhtran/Documents/terminology-extracti...,30.70


In [19]:
df_ls = [adj_noun, noun_adp_noun, noun_adp_det_noun, noun_adp_adj_noun, adp_noun, adj_adj_noun]

for d in df_ls:
    raw_res = raw_res.merge(d, on='DOMAIN', how='left')
raw_res

,DOMAIN,NON_PATTERN,ADJ_NOUN,NOUN_ADP_NOUN,NOUN_ADP_DET_NOUN,NOUN_ADP_ADJ_NOUN,ADP_NOUN,ADJ_ADJ_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,45.76,44.85,44.77,45.76,45.62,43.43,45.57
1,/Users/hanhtran/Documents/terminology-extracti...,51.86,50.61,50.90,51.86,51.70,49.27,51.71
2,/Users/hanhtran/Documents/terminology-extracti...,37.99,35.05,36.20,37.95,37.30,34.15,37.50
3,/Users/hanhtran/Documents/terminology-extracti...,42.35,39.66,40.40,42.33,41.64,38.46,41.83
4,/Users/hanhtran/Documents/terminology-extracti...,24.24,23.90,23.88,24.24,24.27,23.01,24.15
5,/Users/hanhtran/Documents/terminology-extracti...,30.77,30.51,30.32,30.75,30.70,29.69,30.70


In [20]:
raw_res.to_csv('nl_iate_comparison_full.csv')

In [23]:
raw_res['DOMAIN'] = [x.split('/')[-1] for x in raw_res['DOMAIN']]
dt = raw_res.set_index('DOMAIN').T.loc['NON_PATTERN':,:]
cols = ['ann_bert_wind.pkl','nes_bert_wind.pkl','ann_bert_equi.pkl','nes_bert_equi.pkl', 'ann_bert_corp.pkl','nes_bert_corp.pkl']
for col in cols:
    dt['percent_'+col] = round(dt[col]*100/dt[col].iloc[0],2) - 100
dt.filter(regex='percent_')

DOMAIN,percent_ann_bert_wind.pkl,percent_nes_bert_wind.pkl,percent_ann_bert_equi.pkl,percent_nes_bert_equi.pkl,percent_ann_bert_corp.pkl,percent_nes_bert_corp.pkl
NON_PATTERN,0.00,0.00,0.00,0.00,0.00,0.00
ADJ_NOUN,-7.74,-6.35,-1.99,-2.41,-1.40,-0.84
NOUN_ADP_NOUN,-4.71,-4.60,-2.16,-1.85,-1.49,-1.46
NOUN_ADP_DET_NOUN,-0.11,-0.05,0.00,0.00,0.00,-0.06
NOUN_ADP_ADJ_NOUN,-1.82,-1.68,-0.31,-0.31,0.12,-0.23
ADP_NOUN,-10.11,-9.19,-5.09,-4.99,-5.07,-3.51
ADJ_ADJ_NOUN,-1.29,-1.23,-0.42,-0.29,-0.37,-0.23


In [24]:
dt[['percent_ann_bert_corp.pkl','percent_ann_bert_equi.pkl','percent_ann_bert_wind.pkl','percent_nes_bert_corp.pkl','percent_nes_bert_equi.pkl','percent_nes_bert_wind.pkl']]

DOMAIN,percent_ann_bert_corp.pkl,percent_ann_bert_equi.pkl,percent_ann_bert_wind.pkl,percent_nes_bert_corp.pkl,percent_nes_bert_equi.pkl,percent_nes_bert_wind.pkl
NON_PATTERN,0.00,0.00,0.00,0.00,0.00,0.00
ADJ_NOUN,-1.40,-1.99,-7.74,-0.84,-2.41,-6.35
NOUN_ADP_NOUN,-1.49,-2.16,-4.71,-1.46,-1.85,-4.60
NOUN_ADP_DET_NOUN,0.00,0.00,-0.11,-0.06,0.00,-0.05
NOUN_ADP_ADJ_NOUN,0.12,-0.31,-1.82,-0.23,-0.31,-1.68
ADP_NOUN,-5.07,-5.09,-10.11,-3.51,-4.99,-9.19
ADJ_ADJ_NOUN,-0.37,-0.42,-1.29,-0.23,-0.29,-1.23


In [25]:
import matplotlib.pyplot as plt